**1.Upload** **dataset**

In [2]:
!unzip '/content/archive.zip'

Archive:  /content/archive.zip
  inflating: spam.csv                


**2.Import required library**

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM,Activation,Dense,Dropout,Input,Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import pad_sequences
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping


**3.Read dataset and data preprocessing**

In [4]:
import chardet

In [5]:
with open('/content/spam.csv', 'rb') as f:
    enc=chardet.detect(f.read())

In [6]:
df=pd.read_csv('/content/spam.csv', encoding = enc['encoding'])
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


Preprocessing

In [7]:
df.isnull().sum()

v1               0
v2               0
Unnamed: 2    5522
Unnamed: 3    5560
Unnamed: 4    5566
dtype: int64

In [8]:
df.info()
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   v1          5572 non-null   object
 1   v2          5572 non-null   object
 2   Unnamed: 2  50 non-null     object
 3   Unnamed: 3  12 non-null     object
 4   Unnamed: 4  6 non-null      object
dtypes: object(5)
memory usage: 217.8+ KB


,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
5568,ham,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN


In [9]:
df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'],axis=1,inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   v1      5572 non-null   object
 1   v2      5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [10]:
X = df.v2
Y = df.v1
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1) 

In [11]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15)

In [12]:
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = pad_sequences(sequences,maxlen=max_len)

**4.Create model and 5.Add layers(LSTM,Dense-(Hidden layers),Output**

In [13]:
inputs = Input(name='inputs',shape=[max_len])
layer = Embedding(max_words,50,input_length=max_len) (inputs)
layer = LSTM(64) (layer)
layer = Dense(256,name='FC1') (layer)
layer = Activation('relu') (layer)
layer = Dropout (0.5) (layer)
layer = Dense(1,name='out_layer') (layer)
layer = Activation('sigmoid') (layer)
model = Model(inputs=inputs,outputs=layer)

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 150)]             0         
                                                                 
 embedding (Embedding)       (None, 150, 50)           50000     
                                                                 
 lstm (LSTM)                 (None, 64)                29440     
                                                                 
 FC1 (Dense)                 (None, 256)               16640     
                                                                 
 activation (Activation)     (None, 256)               0         
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 out_layer (Dense)           (None, 1)                 257   

**6.Compile the model**

In [14]:
model.compile(optimizer='adam',loss='mse')

**7.Fit the model**

In [15]:
model.fit(sequences_matrix,Y_train,batch_size=128,epochs=10,
          validation_split=0.2)

Epoch 1/10
30/30 [==============================] - 11s 301ms/step - loss: 0.1427 - val_loss: 0.0843
Epoch 2/10
30/30 [==============================] - 9s 288ms/step - loss: 0.0476 - val_loss: 0.0166
Epoch 3/10
30/30 [==============================] - 8s 270ms/step - loss: 0.0112 - val_loss: 0.0119
Epoch 4/10
30/30 [==============================] - 8s 260ms/step - loss: 0.0073 - val_loss: 0.0113
Epoch 5/10
30/30 [==============================] - 8s 260ms/step - loss: 0.0050 - val_loss: 0.0112
Epoch 6/10
30/30 [==============================] - 8s 261ms/step - loss: 0.0037 - val_loss: 0.0112
Epoch 7/10
30/30 [==============================] - 8s 261ms/step - loss: 0.0032 - val_loss: 0.0110
Epoch 8/10
30/30 [==============================] - 8s 261ms/step - loss: 0.0027 - val_loss: 0.0110
Epoch 9/10
30/30 [==============================] - 9s 310ms/step - loss: 0.0026 - val_loss: 0.0105
Epoch 10/10
30/30 [==============================] - 8s 262ms/step - loss: 0.0025 - val_loss: 0.010

**8.Save the model**

In [16]:
model.save('Spam_classifier.h5')

**9.Test the model**

In [17]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = pad_sequences(test_sequences,maxlen=max_len)

In [18]:
accr = model.evaluate(test_sequences_matrix,Y_test)

27/27 [==============================] - 1s 22ms/step - loss: 0.0094
